In [1]:
a = [1, 2]
b = [3, 4]
a+b

[1, 2, 3, 4]

In [8]:
from math import sqrt, prod, log2

def PrimeFactors(n):
    '''list prime factors of n > 1'''
    if n <= 1: return
    f, d = [], 2
    while True:
        if not n % d: f.append(d); return f if n == 2 else f + PrimeFactors(n//d)
        if d > int(sqrt(n)): f.append(n); return f
        d = 3 if d == 2 else d + 2

        
def GCD(a, b):
    '''greatest common divisor of a and b both >= 1'''
    if a < 1 or b < 1: return
    af, bf, gcd = factor(a), factor(b), 1
    for f in af:
        if f in bf: 
            bf.remove(f)
            gcd *= f
    return gcd


def RP(a, b):
    '''relatively prime?'''
    return True if gcd(a, b) == 1 else False


def FactorExp(n):
    '''factorization of n > 1 returned as a list of (prime, exponent) tuples'''
    if n <= 1: return
    f = sorted(PrimeFactors(n))
    p, ptr = [(f[0], 1)], 0
    for i in f[1:]: 
        if i == p[ptr][0]: p[ptr] = (i, p[ptr][1] + 1)
        else: p.append((i, 1)); ptr += 1
    return(p)


def ListProduct(l):
    '''product of all elements of list l: uses built-in math function'''
    return prod(l)


def Divisors(n):
    '''list divisors of n > 0'''
    if n < 1: return
    d, f = [1], PrimeFactors(n)
    nf = len(f)
    for i in range(1, 2**nf):
        k = boolkey(i)
        this_list = [1]
        for j in range(len(k)):
            if k[j]: this_list.append(f[j])
        d.append(ListProduct(this_list))
    return sorted(list(set(d)))


def boolkey(n):
    '''return list of bool values of n in binary'''
    key = [False]*(int(log2(n))+1)
    while n > 0:
        p = int(log2(n))
        key[p] = True
        n -= 2**p
    return key


def Totient(n):
    '''Euler's totient(n) = count n's relative primes 1 < rp < n'''
    if n < 1: return
    totient = 1
    for i in range(2, n):
        if rp(n, i): totient += 1
    return totient


def Mobius(n):
    '''mu(1) = 1. mu(n>1) = 0 or (-1)**k; see ANT chapter 2'''
    if n < 1: return
    if n == 1: return 1
    p = FactorExp(n)
    for i in p:
        if i[1] > 1: return 0
    return (-1)**len(p)


print(Divisors(36000))

# for i in range(1, 12): print(i, totient(i))

# x = 2*3*5*7*11*13
# print(x, factor(x), pfactor(x), mobius(x))
# x = x*17
# print(x, factor(x), pfactor(x), mobius(x))


[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 30, 32, 36, 40, 45, 48, 50, 60, 72, 75, 80, 90, 96, 100, 120, 125, 144, 150, 160, 180, 200, 225, 240, 250, 288, 300, 360, 375, 400, 450, 480, 500, 600, 720, 750, 800, 900, 1000, 1125, 1200, 1440, 1500, 1800, 2000, 2250, 2400, 3000, 3600, 4000, 4500, 6000, 7200, 9000, 12000, 18000, 36000]


In [22]:
def CD(i, n): 
    '''
    Conjugate divisor
    '''
    return n//i

def Nu(n):
    if n == 1: return 0
    return len(PrimeFactors(n))


def Dirichlet(f, g, n):
    '''
    Dirichlet multiplication of two functions f and g for value n
    '''
    if n == 1: return f(1)*g(1)
    total = 0
    for i in Divisors(n):
        # print(i, f(i), g(CD(i, n)))
        total += f(i) * g(CD(i, n))
    return total


total0, total1, totalOther, n = 0, 0, 0, 100000
for i in range(1, n+1): 
    product = Dirichlet(Mobius, Nu, i)
    if product == 0:   total0 += 1
    elif product == 1: total1 += 1
    else:              totalOther += 1
    
    
print(total0, total1, totalOther)



90300 9700 0


In [114]:
from random import randint

start = randint(-10, 10)
current = start
length = randint(10, 20)
pivot = randint(0, length)
data = [1]*length

for i in range(length):
    this_index = i + pivot
    if this_index >= length: this_index -= length
    data[this_index] = current + randint(1,4)
    current = data[this_index]
    

correct_answer = randint(0, length-1)
target = data[correct_answer]

print(data)
print(len(data), 'is length')
print(pivot, 'is the pivot')
print(target, 'is the target')
print(correct_answer, 'is the solution')

[14, -4, -2, 2, 3, 5, 6, 8, 9, 13]
10 is length
1 is the pivot
-2 is the target
2 is the solution


In [115]:
# def FindTargetIndex(l, t): return l.index(t)

# P is an index in pivot space. S is an index in the source list.
#   S = [6, 8, 10, -2, 0, 2, 4]
#   the value 4 has an S-index of 6. P-index is 2 with p = 4

def FindP(d):
    '''
    return the pivot index (start index of ascending data in a ring)
    '''
    safety, safety_limit = 0, 20
    
    i0, i1 = 0, len(d) - 1
    iq = i0 + (i1 - i0)//2
    print(i0, iq, i1, safety)
    while True:
        safety += 1
        if safety > safety_limit: break
        
        iq = i0  + (i1 - i0) // 2
        if d[iq] - d[i0] > 0:
            if d[i0-1] > d[i0]: return i0
            else: i0 = iq
        else: i1 = iq
        
        # print(i0, iq, i1, d[i0], d[iq], d[i1], safety)
        
        if i0 == i1: break
    p = i0 + 1
    if p >= len(d): p -= len(d)
    return p 
    
def P(p, i, l):
    '''convert a pivot-space (p) index i for a data list of length l to a source index s'''
    s = i + p
    if s >= l: s -= l
    if s < 0: s += l
    return s

def FindTargetIndex(d, t): 
    p = FindP(d)
    print('pivot is', p)
    print('target is', t)
    print('correct answer is', d.index(t))
    l = len(d)
    t0, t1 = 0, l - 1
    tq = t1//2
    safety = 0
    safety_limit = 10
    while True:
        
        print(t0, tq, t1)
        
        safety += 1
        if safety == safety_limit: 
            print('uh oh')
            break
        
        tq = t0 + (t1 - t0)//2
        if t < d[P(p, tq, l)]: t1 = tq
        else: t0 = tq
        
        print('                 ...', t0, tq, t1, d[P(p, t0, l)], d[P(p, tq, l)], d[P(p, t1, l)])
        
        if d[P(p, t0, l)] == t: break
    return P(p, t0, l)


my_answer = FindTargetIndex(data, target) 
print('my answer is', my_answer)

0 4 9 0
pivot is 1
target is -2
correct answer is 2
0 4 9
                 ... 0 4 4 -4 5 5
0 4 4
                 ... 0 2 2 -4 2 2
0 2 2
                 ... 1 1 2 -2 -2 2
my answer is 2
